In [50]:
#CODE TO GET PERCENTAGE CHANGES BY TREASURY ID
import pandas as pd
import os

In [51]:
#FEB 2024 CBO SPENDING PROJECTIONS
currentyr = 2024
years = [i for i in range(currentyr, currentyr+11)] 
cbo_ot = pd.read_excel('data/cbo-2024-02-Spending-Projections.xlsx', header=7, sheet_name=1)
cbo_ot = cbo_ot[[col for col in cbo_ot.columns if col not in years]] #remove the BA cols
cbo_ot.columns = cbo_ot.columns.str.replace('.1', '')
cbo_ot = cbo_ot.drop(columns=['Unnamed: 9', "Unnamed: 21", "Off-Budget?"]).drop(index=0).drop(cbo_ot.index[-2:])
cbo_ot['25-34'] = cbo_ot.loc[:, '2025':'2034'].sum(axis=1)

cbo_ot["Agency"] = cbo_ot["Agency"].str.strip()
cbo_ot["Bureau"] = cbo_ot["Bureau"].str.strip()
cbo_ot['Title'] = cbo_ot['Title'].str.strip()

byprogram_24 = cbo_ot[["Treasury Identification Number", "25-34"]].groupby("Treasury Identification Number").sum()
byprogram_24.shape

(1686, 1)

In [52]:
cbo_ot.columns

Index(['Treasury Identification Number', 'Title', 'Discretionary or Mandatory',
       'Major Spending Category', 'Agency', 'Bureau', 'Function',
       'Subfunction', '2024', '2025', '2026', '2027', '2028', '2029', '2030',
       '2031', '2032', '2033', '2034', '25-34'],
      dtype='object')

In [53]:
#FEB 2021 CBO SPENDING PROJECTIONS
cbo21_ot = pd.read_excel('data/cbo-2021-02-11-spendingprojections.xlsx', header=7, sheet_name=1)
years_21 = [i for i in range(2021, 2032)]
cbo21_ot = cbo21_ot[[col for col in cbo21_ot.columns if col not in years_21]].drop(columns=["Off-Budget?", "Unnamed: 9", "Unnamed: 21"]) #remove the BA cols, unnamed and unnecessary columns
cbo21_ot.columns = cbo21_ot.columns.str.replace('.1', '') #clean OT columns
cbo21_ot["22-31"] = cbo21_ot.loc[:, '2022':'2031'].sum(axis=1) #sum the years

byprogram_21 = cbo21_ot[["Treasury Identification Number", "22-31"]].groupby("Treasury Identification Number").sum() #group by agency-bureau-program and sum the total
byprogram_21.head(4)

,22-31
Treasury Identification Number,
000-0000-0-1-373,4951
000-0000-0-1-604,1000
000-0100-0-1-801,381
000-0110-0-1-801,2443


In [54]:
i = 0
y = 0
for tid24 in byprogram_24.index:
    if tid24 not in byprogram_21.index:
        i += 1
for tid21 in byprogram_21.index:
    if tid21 not in byprogram_24.index:
        y += 1
print("Total 2024 TIDs not in 2021 TIDs:", i)
print("Total 2021 TIDs not in 2024 TIDs:", y)

Total 2024 TIDs not in 2021 TIDs: 170
Total 2021 TIDs not in 2024 TIDs: 125


In [55]:
#merge the two dataframes
merged = pd.merge(byprogram_21, byprogram_24, on="Treasury Identification Number", how="right")
merged['percent_change'] = ((merged['25-34'] - merged['22-31']) / abs(merged['22-31'])) * 100
merged.reset_index(inplace=True)
merged

,Treasury Identification Number,22-31,25-34,percent_change
0,000-0100-0-1-801,381.0,351.0,-7.874016
1,000-0110-0-1-801,2443.0,2819.0,15.390913
2,000-0123-0-1-801,241.0,270.0,12.033195
3,000-0126-0-1-801,110.0,193.0,75.454545
4,000-0127-0-1-801,1523.0,1952.0,28.168089
...,...,...,...,...
1681,576-9993-8-2-908,NaN,-570.0,NaN
1682,579-8299-0-7-552,6184.0,3961.0,-35.947607
1683,580-5585-0-2-376,571.0,391.0,-31.523643
1684,581-5577-0-2-376,5924.0,8835.0,49.139095


In [57]:
with_info = pd.merge(merged, cbo_ot[["Treasury Identification Number","Agency", "Bureau", "Title"]], on="Treasury Identification Number", how="left")
with_info.drop_duplicates(keep='first', inplace=True)
with_info = with_info[['Treasury Identification Number', 'Agency', 'Bureau', 'Title', '22-31', '25-34', 'percent_change']]
with_info['22-31'] = with_info['22-31'].fillna('N/A')
with_info['percent_change'] = with_info['percent_change'].fillna('N/A')
with_info

,Treasury Identification Number,Agency,Bureau,Title,22-31,25-34,percent_change
0,000-0100-0-1-801,Legislative Branch,Senate,"Compensation of members, Senate",381.0,351.0,-7.874016
1,000-0110-0-1-801,Legislative Branch,Senate,"Salaries, officers and employees",2443.0,2819.0,15.390913
2,000-0123-0-1-801,Legislative Branch,Senate,Miscellaneous items,241.0,270.0,12.033195
3,000-0126-0-1-801,Legislative Branch,Senate,Secretary of the Senate,110.0,193.0,75.454545
4,000-0127-0-1-801,Legislative Branch,Senate,Sergeant at Arms and Doorkeeper of the Senate,1523.0,1952.0,28.168089
...,...,...,...,...,...,...,...
1945,576-9993-8-2-908,Securities Investor Protection Corporation,Securities Investor Protection Corporation,"Earnings on investments, SIPC",N/A,-570.0,N/A
1946,579-8299-0-7-552,Patient-Centered Outcomes Research Trust Fund,Patient-Centered Outcomes Research Trust Fund,Patient-Centered Outcomes Research Trust Fund,6184.0,3961.0,-35.947607
1947,580-5585-0-2-376,Corporation for Travel Promotion,Corporation for Travel Promotion,Travel Promotion Fund,571.0,391.0,-31.523643
1948,581-5577-0-2-376,Bureau of Consumer Financial Protection,Bureau of Consumer Financial Protection,Bureau of Consumer Financial Protection Fund,5924.0,8835.0,49.139095


In [62]:
non_float_values = with_info[~with_info['percent_change'].apply(lambda x: isinstance(x, float))].query('percent_change != "N/A"')
inf_values = with_info[with_info['percent_change'].isin([float('inf'), float('-inf')])]
inf_values.shape

(14, 7)

In [63]:
with_info = with_info.replace([float('inf'), float('-inf')], float('nan'))
print(with_info.shape)
with_info = with_info.dropna()
print(with_info.shape)

(1686, 7)
(1672, 7)


In [64]:
sheetname = "changes by TIN"
filename = "output/cbo_projection_changes.xlsx"
df = with_info
with pd.ExcelWriter(filename, mode='a', engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name=sheetname, index=False)